<a href="https://colab.research.google.com/github/joshid43016/AnalyticsDataEcosystem/blob/main/sentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.4 MB/s eta 0:14:10tcmalloc: large alloc 1147494400 bytes == 0x55c533a94000 @  0x7f833b624615 0x55c4faf084cc 0x55c4fafe847a 0x55c4faf0b2ed 0x55c4faffce1d 0x55c4faf7ee99 0x55c4faf799ee 0x55c4faf0cbda 0x55c4faf7ed00 0x55c4faf799ee 0x55c4faf0cbda 0x55c4faf7b737 0x55c4faffdc66 0x55c4faf7adaf 0x55c4faffdc66 0x55c4faf7adaf 0x55c4faffdc66 0x55c4faf7adaf 0x55c4faf0d039 0x55c4faf50409 0x55c4faf0bc52 0x55c4faf7ec25 0x55c4faf799ee 0x55c4faf0cbda 0x55c4faf7b737 0x55c4faf799ee 0x55c4faf0cbda 0x55c4faf7a915 0x55c4faf0cafa 0x55c4faf7ac0d 0x55c4faf799ee
     |█████████████████               | 1055.7 MB 1.4 MB/s eta 0:11:21tcmalloc: large alloc 1434370048 bytes == 0x55c5780ea000 @  0x7f833b624615 0x55c4faf084cc 0x55c4fafe847a 0x55c4faf0b2ed 0x55c4faffce1d 0x55c4faf7ee99 0x55c4faf799ee 0x55c4faf0cbda 0x55c4faf7ed00 0x55c4faf799ee 0x55c4faf0cbda 0x55c4faf7b737 0x55c4faffdc66 0x55c4faf7ad

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 3.3 MB 6.6 MB/s 
     |████████████████████████████████| 61 kB 525 kB/s 
     |████████████████████████████████| 596 kB 71.4 MB/s 
     |████████████████████████████████| 3.3 MB 24.5 MB/s 
     |████████████████████████████████| 895 kB 56.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [5]:
#Pretrained NLP model from hugging face
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [20]:
tokens = tokenizer.encode('It was good  could have been great', return_tensors='pt') 
# pt for pytorch 
result = model(tokens)
result.logits
int(torch.argmax(result.logits))+1

3

In [16]:
#tokens
#tokenizer.decode(tokens[0])
#result
# Values are 1 to 5 with 5 being most positive 
result.logits


tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward>)

In [25]:
# Collect reviews 
r = requests.get('https://www.yelp.com/biz/the-eco-plumbers-hilliard-2?osq=Plumbers')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]
reviews

['This is not your normal plumbing company. These guys are the best. From their customer service, to their advanced scheduling notifications with pictures of who is coming to your house & when! \xa0I have used the Eco Plumbers for years and they are the only plumbers I trust with my house in the Columbus area.',
 'Hi Joseph! Thank you for putting your trust in our company. We always strive to provide the highest quality of service and are happy to hear you had another wonderful experience with us. Your satisfaction is very important to our team and we were pleased to assist! We look forward to your call should you need us in the future. Have a wonderful day, The Eco Plumbers.',
 "I have had them out for several all services at this point. \xa0My first service with them was because I had a shower head that was leaking in the wall. They were able to easily fix it for me. I then had a gas line problem to my fireplace and they were able to fix the line and get it hooked up so the on/off va

In [27]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [30]:
import numpy as np
import pandas as pd
df = pd.DataFrame(np.array(reviews), columns=['review'])
df['review'].iloc[0]

df.head (5) 

,review
0,This is not your normal plumbing company. Thes...
1,Hi Joseph! Thank you for putting your trust in...
2,I have had them out for several all services a...
3,"Hello Megan, we appreciate your business and a..."
4,"My toilets, shower and tub were backing up whe..."


In [32]:
sentiment_score(df['review'].iloc[2])


5

In [33]:
# Run on all reviews. lambda to loop through 
# Only 512 first token 
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
df
 

,review,sentiment
0,This is not your normal plumbing company. Thes...,5
1,Hi Joseph! Thank you for putting your trust in...,5
2,I have had them out for several all services a...,5
3,"Hello Megan, we appreciate your business and a...",5
4,"My toilets, shower and tub were backing up whe...",5
5,Hi Maria! Thank you for your feedback and this...,5
6,Called on a Friday afternoon to receive a main...,1
7,"Hi Christina, we are sorry that you did not ex...",5
8,Constant up-selling and overpricing. The guy w...,2
9,Hi Vel! Thank you for your feedback. I have pa...,5


In [34]:
df['review'].iloc[17]

"I never ever write reviews but this has me so upset I have to, 3 months ago I hade Eco Plumbers install 2 hot water tanks and quite frankly was pleased, knowing how I felt about the previous experience with them I decided to call and have them reinstall a kitchen faucet , 1/2 bath faucet and disposal with my own fixtures. I was willing to pay a fare price however I was shocked when they came to my house and gave me a price of almost $1300.00 ( with a straight face) Again I was supplying all the items that were to be installed except for 2 supply lines that they wanted $186.00 for I bought \xa0the same supply lines for $12.00 at Menards. I now feel totally used by this company about my hot water tanks ,( How much did I over pay for those).I can't tell you not to use them but I will say they seem to try fool you into using them with cute commercials but their prices are very ugly."